In [18]:
#import
import os
import sys
import subprocess

from kaleido.scopes.base import which_py2

spgo =  subprocess.getoutput
import ants

opj = os.path.join
opb = os.path.basename
opd = os.path.dirname
ope = os.path.exists
opi = os.path.isfile

# Where to find whatever you need ################################################################################
#cmd ='hostname'

if os.uname()[1] == 'ClaClaBoss':
    path_code  = opj('/', 'media', 'simon', 'LaCie', 'travail', 'simon', 'current_works', 'pipeline','ucbl','code','v202511')
    path_ATLAS = opj('/', 'media', 'simon', 'LaCie', 'travail', 'simon', 'fMRI', 'MRI_atlas', 'atlas')
    singularity = 'no'
elif os.uname()[1] == 'Dhanistha':
    path_code = opj('/', 'media', 'clavagni', 'LaCie', 'travail', 'simon', 'current_works', 'pipeline','ucbl','code','v202511')
    path_ATLAS = opj('/', 'media', 'clavagni', 'LaCie', 'travail', 'simon', 'fMRI', 'MRI_atlas', 'atlas')
    singularity = 'no'
elif os.uname()[1] == 'gpu-05-01':
    Major_path = opj('/', 'srv', 'projects')
    path_ATLAS = opj(Major_path, 'tremblaybgtemplate', 'Atlas', )
    singularity = 'yes'

##############################
study  = 'test'
specie = 'Human'
#############################

sys.path.insert(1, path_code)

from tools import run_cmd,forsing
from anat import setupFS,WB_label,WB_matrices,prepro,mask_seg_label,norm2template,preFS,FS_surf,FS_freeview,FS_flat,FS_finalise,FS_2_WB

# To set :
Subject_Dir          = 'collin27'

main_path =  opj('/','home','clavagni','Documents','todo')

dir_path = opj(main_path,Subject_Dir,'anat')
template   = 'S1200'                       # BAL2013, MY19,NMT or MEBRAIN
list_atlas = [['EDNIxCSCLR', 'EDNIxCSC'],['ctab', 'txt'],[4, 4],[1, 1]]                        # 0:No / 1:Yes
do_Nu      = [0]
diary_name = opj(dir_path,'dairy.txt')

sing_afni, sing_fsl, sing_fs, sing_wb, sing_itk, _ = forsing.get(main_path, study, singularity, path_ATLAS, main_path)

# Freesurfer set up
FS_dir, fs_tools, cmd_tksurfer, cmd_flatten, cmd_mris, export_fs = setupFS.get(main_path,dir_path,study,path_code,sing_fs)

second_path= opj(path_ATLAS,'mammals','primates',specie)
path_code_label = opj(second_path,'label_code')



/usr/local/freesurfer/7.3.2
/home/clavagni/Documents/todo/collin27/anat/freesurfer


In [3]:

cmd = (sing_fs + 'mri_convert ' + opj(FS_dir,Subject_Dir,'mri','brainmask.mgz') +
       ' ' +  opj(dir_path,'native','volumes','masks',Subject_Dir + '_mask.nii.gz'))
run_cmd.run(cmd,diary_name)

msk = ants.image_read(opj(dir_path,'native','volumes','masks',Subject_Dir + '_mask.nii.gz'))
msk[msk>0]=1
ants.image_write(msk,opj(dir_path,'native','volumes','masks',Subject_Dir + '_mask.nii.gz'))


mri_convert /home/clavagni/Documents/todo/collin27/anat/freesurfer/collin27/mri/brainmask.mgz /home/clavagni/Documents/todo/collin27/anat/native/volumes/masks/collin27_mask.nii.gz 
reading from /home/clavagni/Documents/todo/collin27/anat/freesurfer/collin27/mri/brainmask.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /home/clavagni/Documents/todo/collin27/anat/native/volumes/masks/collin27_mask.nii.gz...



In [9]:
imgfile = opj(dir_path,'native','volumes',Subject_Dir + '_space-acpc_desc-SS_T1w.nii.gz')
img = ants.image_read(imgfile)

reffile = opj(dir_path,'native','volumes','MNI152_space-acpc_desc-SS2_T1w.nii.gz')
ref =  ants.image_read(reffile)
msk = ants.image_read(opj(dir_path,'native','volumes','MNI152_T1_1mm_brain_mask_strict.nii.gz'))
refmsk = ref*msk

mTx6 = ants.registration(fixed=refmsk, moving=img,
                             type_of_transform='SyNCC',
                             initial_transform=None,
                             outprefix=opj(dir_path,'matrices','anat2'+ template + '_SyN_'))


new_anat = ants.apply_transforms(refmsk,img,transformlist=mTx6['fwdtransforms'],interpolator='hammingWindowedSinc')
ants.image_write(new_anat,opj(dir_path,'templates',template,'volumes',Subject_Dir + '_space-'+ template +'_desc-SS_T1w.nii.gz'))

for i in ['EDNIxCSCLR','EDNIxCSC']:
    label = opj('/media/clavagni/LaCie/travail/simon/fMRI/MRI_atlas/ednix/label_code',i+'_label.txt')
    atlas1= ants.image_read('/media/clavagni/LaCie/travail/simon/fMRI/MRI_atlas/ednix/atlas/mammals/primates/Human/volumes/labels/Human_seg-' + i + '_dseg.nii.gz')
    new_atlas = ants.apply_transforms(img,atlas1,
                                      transformlist=[opj(dir_path,'matrices','anat2'+ template + '_SyN_1InverseWarp.nii.gz'),
                                                     opj(dir_path,'matrices','anat2'+ template + '_SyN_0GenericAffine.mat')],
                                      interpolator='genericLabel',whichtoinvert=[False,True],
                                      imagetype=3)
    ants.image_write(new_atlas,opj(dir_path,'native','volumes','labels',Subject_Dir + '_seg-'+ i + '_dseg.nii.gz'))
    cmd = (sing_wb + 'wb_command -volume-label-import' +
           ' ' + opj(dir_path,'native','volumes','labels',Subject_Dir + '_seg-'+ i + '_dseg.nii.gz') +
           ' ' + label +
           ' ' + opj(dir_path,'native','volumes','labels',Subject_Dir + '_seg-'+ i + '_dseg.nii.gz') +
           ' -drop-unused-labels')
    run_cmd.run(cmd,diary_name)



In [ ]:

cmd =(sing_fs + 'mris_volmask --aseg_name aseg --label_left_white 2 --label_left_ribbon 3 --label_right_white 41 --label_right_ribbon 42 --save_ribbon ' + Subject_Dir)
run_cmd.do(cmd,diary_name)

os.remove(opj(FS_dir,Subject_Dir,'mri', 'lh.ribbon.mgz'))
os.remove(opj(FS_dir, Subject_Dir, 'mri', 'rh.ribbon.mgz'))

In [12]:
a = list()
for x in list_atlas[0]:
    a.append(opj(dir_path,'native','volumes','labels',Subject_Dir + '_seg-'+ x + '_dseg.nii.gz'))
print(a)
for i,j,k in zip(a,list_atlas[1],list_atlas[3]):
    if opi(i)==True:
        if k == 1:
            WB_label.surfFS(i, j, 0, diary_name,sing_fs,path_code_label,export_fs)

embedding color table /media/clavagni/LaCie/travail/simon/fMRI/MRI_atlas/atlas/mammals/primates/Human/label_code/EDNIxCSC_StatsLUT.txt into output annot file
using mid as surface name
projecting 0.01 % along surface normal
reading parcellation volume from /home/clavagni/Documents/todo/collin27/anat/freesurfer/collin27/mri/collin27_EDNIxCSC4.mgz...
reading input surface /home/clavagni/Documents/todo/collin27/anat/freesurfer/collin27/surf/rh.mid...
pass 1: 258 segments changed
pass 2: 7 segments changed
pass 3: 5 segments changed
pass 4: 5 segments changed
pass 5: 5 segments changed
pass 6: 5 segments changed
pass 7: 5 segments changed
pass 8: 5 segments changed
pass 9: 5 segments changed
pass 10: 5 segments changed
pass 11: 5 segments changed
writing annotation to rh.collin27_EDNIxCSC4.annot...
embedding color table /media/clavagni/LaCie/travail/simon/fMRI/MRI_atlas/atlas/mammals/primates/Human/label_code/EDNIxCSC_StatsLUT.txt into output annot file
using mid as surface name
projecting 

In [13]:
WB_matrices.ants2wb(opj(dir_path,'matrices','anat2'+ template + '_SyN_0GenericAffine.mat'),opj(dir_path,'matrices'),template,opj(dir_path,'diary.txt'),sing_wb)

In [16]:
FS_2_WB.WB_prep(cmd_mris,FS_dir,opj(fs_tools,'standard_mesh_atlases'),dir_path,Subject_Dir ,specie,'','',template,list_atlas,opj(second_path,'atlas'),'',0,opj(dir_path,'diary.txt'),'','','','')

TypeError: 'int' object is not subscriptable

In [14]:
new_anat = ants.apply_transforms(img,refmsk,transformlist=mTx6['invtransforms'],whichtoinvert=[True,False], interpolator='hammingWindowedSinc')
ants.image_write(new_anat,opj(dir_path,'templates',template,'volumes',template + '_space-'+ Subject_Dir +'_desc-SS_T1w.nii.gz'))

In [17]:
from anat import preFS
preFS.msk_RS(Subject_Dir,opj(dir_path,'native','volumes','masks',Subject_Dir + '_mask.nii.gz'),opj(dir_path,'native','volumes','labels',Subject_Dir + '_seg-4FS_dseg.nii.gz'),diary_name)


Done
